In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# One-Hot Embeddings: Bag-of-Words Classifier

In [2]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH"),
        ("Yo creo que si".split(), "SPANISH"),
        ("it is lost on me".split(), "ENGLISH")]

word2idx = {}
for sent, _ in data:
    for word in sent:
        if word not in word2idx:
            word2idx[word] = len(word2idx)
print(word2idx)

label2idx = {"SPANISH": 0, "ENGLISH": 1}

VOC_SIZE = len(word2idx)
N_LABEL = len(label2idx)

X = []
y = []
for sent, label in data:
    # It would create double by default
    vec = np.zeros(VOC_SIZE)
    for word in sent:
        vec[word2idx[word]] += 1
    X.append(vec)
    y.append(label2idx[label])

# X -> torch.float32
# y MUST BE torch.int64 (long tensor)
X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)
print(X)
print(y)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}
tensor([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
         1., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 1.

In [3]:
class BoWClassifier(nn.Module):
    def __init__(self, n_label, voc_size):
        # All units with differentiable parameters should be included in __init__ function
        # non-linearity function like tanh, ReLU and softmax without differentiable parameters
        # could not be included in __init__ function
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(voc_size, n_label)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=-1)

In [4]:
model = BoWClassifier(N_LABEL, VOC_SIZE)

for param in model.parameters():
    print(param)

bow_vec = X[0]
log_probs = model(bow_vec)
print(log_probs)
print(log_probs.size())

Parameter containing:
tensor([[ 0.0529, -0.0938,  0.0649,  0.0829,  0.0549, -0.1317, -0.0619, -0.1921,
         -0.0792, -0.0732,  0.1279, -0.1948,  0.0977,  0.0807, -0.0855,  0.0725,
          0.0597,  0.0027, -0.0096, -0.1503, -0.0733,  0.0118,  0.0766, -0.1453,
         -0.1289,  0.1942],
        [-0.0320, -0.0091,  0.1817,  0.1826, -0.1503,  0.1631,  0.1925,  0.0114,
          0.0593,  0.0353,  0.1731, -0.1065, -0.0279, -0.0096, -0.1267, -0.0488,
          0.0112,  0.1951, -0.1623, -0.0844,  0.1522,  0.1213, -0.1682,  0.1633,
          0.1182, -0.0768]], requires_grad=True)
Parameter containing:
tensor([0.1168, 0.0356], requires_grad=True)
tensor([-0.8117, -0.5872], grad_fn=<LogSoftmaxBackward>)
torch.Size([2])


In [5]:
train_X, test_X = X[:4], X[4:]
train_y, test_y = y[:4], y[4:]

# Prediction result before training. 
for bow_vec, label in zip(test_X, test_y):
    log_probs = model(bow_vec)
    print(log_probs, label)

# The weight vector for specific word
# The weight vector works like an embedding vector, mapping a word (i.e., one-hot embedding vector) 
# to a dense vector, except for the bias. 
print(model.linear.weight[:, word2idx['creo']])

tensor([-1.0427, -0.4346], grad_fn=<LogSoftmaxBackward>) tensor(0)
tensor([-0.6106, -0.7832], grad_fn=<LogSoftmaxBackward>) tensor(1)
tensor([0.1279, 0.1731], grad_fn=<SelectBackward>)


In [6]:
# Training
# Negative log-likelihood
loss_func = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    running_loss = 0.0
    for bow_vec, label in zip(train_X, train_y):
        # log_probs MUST BE a matrix to enter the loss function
        # use view-function to reshape!!
        log_probs = model(bow_vec.view(1, -1))
        
        # log_probs -> matrix
        # label -> vector
        loss = loss_func(log_probs, label.view(1))
        running_loss += loss.item()

        # Backward propagation
        model.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
            print(epoch + 1, running_loss)

10 0.3749711625277996
20 0.18750330992043018
30 0.12462067138403654
40 0.09322894643992186
50 0.07443659007549286
60 0.061934396624565125
70 0.05302054854109883
80 0.04634522134438157
90 0.04116053646430373
100 0.037017352879047394


In [7]:
# Prediction result after training. 
for bow_vec, label in zip(test_X, test_y):
    log_probs = model(bow_vec)
    print(log_probs, label)

# The weight vector for specific word
# The weight vector works like an embedding vector, mapping a word (i.e., one-hot embedding vector) 
# to a dense vector, except for the bias. 
print(model.linear.weight[:, word2idx['creo']])

tensor([-0.2483, -1.5146], grad_fn=<LogSoftmaxBackward>) tensor(0)
tensor([-2.2676, -0.1093], grad_fn=<LogSoftmaxBackward>) tensor(1)
tensor([ 0.5366, -0.2356], grad_fn=<SelectBackward>)


# Dense Embeddings
Encode semantic similarity in words. 

In [8]:
word2idx = {'hello': 0, 'world': 1}
emb = nn.Embedding(2, 5)
print(emb.weight)

# Indexing-input MUST BE torch.long/torch.int64
t = torch.tensor(word2idx['hello'], dtype=torch.long)
t_emb = emb(t)
print(t_emb)

Parameter containing:
tensor([[-0.4127,  0.6977, -2.0971,  1.0450,  0.8259],
        [-0.1653,  0.5448, -0.3342,  2.5330, -0.9653]], requires_grad=True)
tensor([-0.4127,  0.6977, -2.0971,  1.0450,  0.8259],
       grad_fn=<EmbeddingBackward>)


In [9]:
# Input as a sequence
idx_seq = torch.tensor([0, 1, 0], dtype=torch.long)
embedded = emb(idx_seq)
embedded

tensor([[-0.4127,  0.6977, -2.0971,  1.0450,  0.8259],
        [-0.1653,  0.5448, -0.3342,  2.5330, -0.9653],
        [-0.4127,  0.6977, -2.0971,  1.0450,  0.8259]],
       grad_fn=<EmbeddingBackward>)

In [10]:
# Input as a minibatch of sequences
idx_seq_batch = torch.tensor([[0, 1], [1, 1], [0, 0]], dtype=torch.long)
embedded = emb(idx_seq_batch)
embedded

tensor([[[-0.4127,  0.6977, -2.0971,  1.0450,  0.8259],
         [-0.1653,  0.5448, -0.3342,  2.5330, -0.9653]],

        [[-0.1653,  0.5448, -0.3342,  2.5330, -0.9653],
         [-0.1653,  0.5448, -0.3342,  2.5330, -0.9653]],

        [[-0.4127,  0.6977, -2.0971,  1.0450,  0.8259],
         [-0.4127,  0.6977, -2.0971,  1.0450,  0.8259]]],
       grad_fn=<EmbeddingBackward>)

# Training Embeddings: N-Gram Model 
In an n-gram language model, given a sequence of words $w$, we want to compute  
$$
P \left( w_{i} \left| w_{i-1}, w_{i-2}, ..., w_{i-n+1} \right. \right)  
$$  
where $w_{i}$ is the i-th word of the sequence.  

In [11]:
# Use TWO words to predict next word
CONTEXT_SIZE = 2
EMB_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
print(trigrams[:3])

voc = set(test_sentence)
word2idx = {word: i for i, word in enumerate(voc)}
VOC_SIZE = len(voc)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [12]:
class NGramModel(nn.Module):
    def __init__(self, voc_size, emb_dim, context_size):
        super(NGramModel, self).__init__()
        self.emb = nn.Embedding(voc_size, emb_dim)
        self.fc1 = nn.Linear(context_size*emb_dim, 128)
        self.fc2 = nn.Linear(128, voc_size)
        
    def forward(self, ins):
        # ins include several words (N=context_size)
        # self.emb(ins) -> (context_size, emb_dim)
        # self.emb(ins).view((1, -1)) -> (1, context_size*emb_dim)
        emb_ins = self.emb(ins).view((1, -1))
        outs = F.relu(self.fc1(emb_ins))
        outs = self.fc2(outs)
        log_probs = F.log_softmax(outs, dim=-1)
        return log_probs

In [13]:
loss_func = nn.NLLLoss()
model = NGramModel(VOC_SIZE, EMB_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        context_idxes = [word2idx[w] for w in context]
        context_var = torch.tensor(context_idxes, dtype=torch.long)
        target_var = torch.tensor([word2idx[target]], dtype=torch.long)
        
        model.zero_grad()
        log_probs = model(context_var)
        
        # log_probs -> matrix
        # target_var -> vector
        loss = loss_func(log_probs, target_var)        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(epoch, total_loss)

0 519.4025988578796
1 517.1042060852051
2 514.8177926540375
3 512.5443403720856
4 510.2832283973694
5 508.03224205970764
6 505.7923231124878
7 503.5623707771301
8 501.3421595096588
9 499.13040494918823


# Computing Word Embeddings: Continuous Bag-of-Words
In an continuous BOW model, given a sequence of words $w$, we want to compute  
$$
P \left( w_{i} \left| w_{i+n-1}, ..., w_{i+1}, w_{i-1}, ..., w_{i-n+1} \right. \right)  
$$  
where $w_{i}$ is the i-th word of the sequence.  

In [14]:
CONTEXT_SIZE = 2
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

voc = set(raw_text)
VOC_SIZE = len(voc)
word2idx = {word: i for i, word in enumerate(voc)}

data = [([raw_text[i-2], raw_text[i-1], raw_text[i+1], raw_text[i+2]], raw_text[i]) for i in range(2, len(raw_text)-2)]
print(data[:3])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study')]


In [15]:
class CBOW(nn.Module):
    def __init__(self, voc_size, emb_dim):
        super(CBOW, self).__init__()
        self.emb = nn.Embedding(voc_size, emb_dim)
        self.fc = nn.Linear(emb_dim, voc_size)
        
    def forward(self, ins):
        emb_ins = self.emb(ins)
        outs = self.fc(emb_ins.sum(dim=0, keepdim=True))
        log_probs = F.log_softmax(outs, dim=-1)
        return log_probs

In [16]:
loss_func = nn.NLLLoss()
model = CBOW(VOC_SIZE, EMB_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in data:
        context_idxes = [word2idx[w] for w in context]
        context_var = torch.tensor(context_idxes, dtype=torch.long)
        target_var = torch.tensor([word2idx[target]], dtype=torch.long)
        
        model.zero_grad()
        log_probs = model(context_var)
        loss = loss_func(log_probs, target_var)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print(epoch, total_loss)

0 250.46009397506714
1 247.94269633293152
2 245.45749354362488
3 243.00389277935028
4 240.58131992816925
5 238.1892442703247
6 235.82717967033386
7 233.49463975429535
8 231.191211104393
9 228.91648137569427
